In [1]:
import os
os.chdir(r'G:\Uzair_OpenEnded') 
from stlf_torch_kit import  DataLoadeing
import torch
import pandas as pd
import numpy as np
import pickle, time
from stlf_torch_kit import univariate_multi_step
from stlf_torch_kit import SaveBestModel, PlotLossCurves, LoadModel, train, TestModel, BatchGenerator, results
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

# Data Loading

In [4]:
import os
path_dataset =r'G:\Uzair_OpenEnded\Dataset'
path_tr = os.path.join(path_dataset, 'CityLearn_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'CityLearn_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'Citylearn_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((6132, 29), (1752, 29), (876, 29))

In [5]:
#time_steps=24 #look back or sequence length, lag, window size #Edit
time_steps = 24
target_len = 1 #how much steps do you wana forecast #Edit
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=target_len)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=target_len)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=target_len)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.0746457576751709 sec


In [4]:
test_X.shape

(851, 24, 29)

#### RNN_MODEL

In [6]:
class RNNModel(nn.Module):
    def __init__(self, time_steps=24, num_features=29):
        super(RNNModel, self).__init__()
        
        self.rnn1 = nn.RNN(input_size=num_features, hidden_size=8, batch_first=True)
        self.rnn2 = nn.RNN(input_size=8, hidden_size=20, batch_first=True)
        
        self.flatten = nn.Flatten()
        self.output_layer = nn.Linear(20, 1)

    def forward(self, x):
        # x shape: (batch_size, time_steps, num_features)
        x, _ = self.rnn1(x)
        x, _ = self.rnn2(x)
        
        # Take the output at the last time step (like Keras default behavior)
        x = x[:, -1, :]  # shape: (batch_size, 20)
        
        x = self.flatten(x)
        output = self.output_layer(x)
        
        return output

# instances

In [7]:
model = RNNModel()#Edit
criterion = nn.MSELoss() #Edit, don't change

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

# Learning Rate & Optimizer

In [8]:
#lr=0.001 # Edit
#optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Edit
lr = 0.001  # Initial learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)  # Added


# Check Device

In [9]:
def get_model_device(model):
    return next(model.parameters()).device
device = get_model_device(model)
print("Model is on device:", device)

Model is on device: cpu


# Training

#### Path & other Stuffs

In [11]:
start_epoch = 1
num_epochs = 20 #Edit
best_model_path=r'G:\Uzair_OpenEnded\chk'+str('\\') #Edit
fig_path=r'G:\Uzair_OpenEnded\chk' #Edit
train_data_loader, validation_data_loader, test_data_loader = DataLoadeing(train_X ,
                                                                           train_y, 
                                                                           validation_X, 
                                                                           validation_y, 
                                                                           test_X, 
                                                                           test_y, 
                                                                           batch_size=32) #Batch_Size Edit

#### Instances

In [12]:
criterion = nn.MSELoss() # Edit, for Now Don't  Change

criterion_mae = nn.L1Loss()

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

#### Training Loop

In [13]:
start = time.time()
train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader)
print('Time Consumed', time.time()-start, "sec")

Epoch [1/20], Step [191/191], Training Loss: 0.0532
Epoch [1/20], Step [54/54], Val Loss: 0.0469

Saving best model for epoch: 1
 at G:\Uzair_OpenEnded\chk\1best_model.pth
Epoch [2/20], Step [191/191], Training Loss: 0.0421
Epoch [2/20], Step [54/54], Val Loss: 0.0454

Saving best model for epoch: 2
 at G:\Uzair_OpenEnded\chk\2best_model.pth
Epoch [3/20], Step [191/191], Training Loss: 0.0401
Epoch [3/20], Step [54/54], Val Loss: 0.0439

Saving best model for epoch: 3
 at G:\Uzair_OpenEnded\chk\3best_model.pth
Epoch [4/20], Step [191/191], Training Loss: 0.0384
Epoch [4/20], Step [54/54], Val Loss: 0.0422

Saving best model for epoch: 4
 at G:\Uzair_OpenEnded\chk\4best_model.pth
Epoch [5/20], Step [191/191], Training Loss: 0.0367
Epoch [5/20], Step [54/54], Val Loss: 0.0400

Saving best model for epoch: 5
 at G:\Uzair_OpenEnded\chk\5best_model.pth
Epoch [6/20], Step [191/191], Training Loss: 0.0346
Epoch [6/20], Step [54/54], Val Loss: 0.0373

Saving best model for epoch: 6
 at G:\Uzai

#### Results

In [14]:
load_model_path=r'G:\Uzair_OpenEnded\chk\20best_model.pth' # Edit
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

# MAPE, MAE, RMSE

New lr = 0.001
Time Consumed 0.08980703353881836 sec
Mean Absolute Error (MAE): 0.39
Median Absolute Error (MedAE): 0.29
Mean Squared Error (MSE): 0.27
Root Mean Squared Error (RMSE): 0.51
Mean Absolute Percentage Error (MAPE): 54.78 %
Median Absolute Percentage Error (MDAPE): 37.64 %


y_test_unscaled.shape=  (851, 1)
y_pred.shape=  (851, 1)
